## quadtree

https://www.cs.princeton.edu/courses/archive/fall03/cs126/assignments/barnes-hut.html
http://www-inf.telecom-sudparis.eu/COURS/CSC5001/new_site/Supports/Projet/NBody/barnes_86.pdf  
https://en.wikipedia.org/wiki/Barnes%E2%80%93Hut_simulation

https://github.com/rdeits/RegionTrees.jl

https://github.com/JuliaArrays/StaticArrays.jl

In [74]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1273


In [1]:
mutable struct Cell
    x ::AbstractFloat
    y ::AbstractFloat
    w ::AbstractFloat
    h ::AbstractFloat
    nbr_of_points ::Int
    sum_of_points_x ::Union{Nothing, AbstractFloat}
    sum_of_points_y ::Union{Nothing, AbstractFloat}
    childrens ::Union{Nothing, Tuple{Cell, Cell, Cell, Cell}}
end;


1. If node x does not contain a body, put the new body b here.
2. If node x is an internal node, update the center-of-mass and total mass of x. Recursively insert the body b in the appropriate quadrant.
3. If node x is an external node, say containing a body named c, then there are two bodies b and c in the same region. Subdivide the region further by creating four children. Then, recursively insert both b and c into the appropriate quadrant(s). Since b and c may still end up in the same quadrant, there may be several subdivisions during a single insertion. Finally, update the center-of-mass and total mass of x. 

In [2]:
function insert!(cell, point_x, point_y)
    # check if node in cell? ... no
    
    # if empty
    if cell.nbr_of_points == 0
        cell.nbr_of_points = 1
        cell.sum_of_points_x = point_x
        cell.sum_of_points_y = point_y
        
    # if one point only
    elseif cell.nbr_of_points == 1
        split!(cell)
        cell.nbr_of_points = 2
        previous_point_x = cell.sum_of_points_x
        previous_point_y = cell.sum_of_points_y
        cell.sum_of_points_x = point_x + previous_point_x
        cell.sum_of_points_y = point_y + previous_point_y
        
        # continue insertion of the point
        quadrant = which_quadrant(point_x, point_y, cell.x, cell.y, cell.w, cell.h)
        insert!(cell.childrens[quadrant], point_x, point_y)

        # insert the old point too
        previousquadrant = which_quadrant(previous_point_x, previous_point_y,
                                          cell.x, cell.y, cell.w, cell.h)
        insert!(cell.childrens[previousquadrant], previous_point_x, previous_point_y)

    else
        cell.sum_of_points_x += point_x
        cell.sum_of_points_y += point_y
        cell.nbr_of_points += 1
        
        quadrant = which_quadrant(point_x, point_y, cell.x, cell.y, cell.w, cell.h)
        insert!(cell.childrens[quadrant], point_x, point_y)
    end
            
    end;

In [3]:
function which_quadrant(x, y, cell_x, cell_y, width, height)
    quadrant = 1
    if x > cell_x + width/2
        quadrant += 1
        end;
    if y < cell_y + height/2
        quadrant += 2
        end;
    return quadrant
    end;
    
# Numbering:
#   1 | 2
#  -------
#   3 | 4
# cell corner (x, y) is bottom left

In [4]:
println( which_quadrant(.1, .1, 0, 0, 1, 1) )
println( which_quadrant(.7, .1, 0, 0, 1, 1) )
println( which_quadrant(.1, .7, 0, 0, 1, 1) )
println( which_quadrant(.8, .8, 0, 0, 1, 1) )

3
4
1
2


In [5]:
println( which_quadrant([.4 .4]..., 0, 0, 1, 1) )

3


In [7]:
function split!(cell)
    half_h = cell.h/2
    half_w = cell.w/2
    cell.childrens = (
        Cell(cell.x, cell.y + half_h, half_w, half_h, 0, nothing, nothing, nothing),
        Cell(cell.x + half_w, cell.y + half_h, half_w, half_h, 0, nothing, nothing, nothing),
        Cell(cell.x, cell.y, half_w, half_h, 0, nothing, nothing, nothing),
        Cell(cell.x + half_w, cell.y, half_w, half_h, 0, nothing, nothing, nothing)
    )
    end;

In [8]:
root = Cell(0.0, 0.0, 1.0, 1.0, 0, nothing, nothing, nothing)

Cell(0.0, 0.0, 1.0, 1.0, 0, nothing, nothing, nothing)

In [9]:
insert!(root, .3, .3)

0.3

In [10]:
root

Cell(0.0, 0.0, 1.0, 1.0, 1, 0.3, 0.3, nothing)

In [11]:
insert!(root, .8, .4)

0.3

In [12]:
root.sum_of_points_x, root.sum_of_points_y

(1.1, 0.7)

In [13]:
insert!(root, .1, .7)

0.7

In [14]:
insert!(root, .42, .34)

0.34

In [15]:
root

Cell(0.0, 0.0, 1.0, 1.0, 4, 1.62, 1.74, (Cell(0.0, 0.5, 0.5, 0.5, 1, 0.1, 0.7, nothing), Cell(0.5, 0.5, 0.5, 0.5, 0, nothing, nothing, nothing), Cell(0.0, 0.0, 0.5, 0.5, 2, 0.72, 0.64, (Cell(0.0, 0.25, 0.25, 0.25, 0, nothing, nothing, nothing), Cell(0.25, 0.25, 0.25, 0.25, 2, 0.72, 0.64, (Cell(0.25, 0.375, 0.125, 0.125, 0, nothing, nothing, nothing), Cell(0.375, 0.375, 0.125, 0.125, 0, nothing, nothing, nothing), Cell(0.25, 0.25, 0.125, 0.125, 1, 0.3, 0.3, nothing), Cell(0.375, 0.25, 0.125, 0.125, 1, 0.42, 0.34, nothing))), Cell(0.0, 0.0, 0.25, 0.25, 0, nothing, nothing, nothing), Cell(0.25, 0.0, 0.25, 0.25, 0, nothing, nothing, nothing))), Cell(0.5, 0.0, 0.5, 0.5, 1, 0.8, 0.4, nothing)))

In [89]:
get_distance(x, y, u, v) = sqrt( (x - u)^2 + (y - v)^2 );

In [90]:
# evaluate cell, x, y, theta=.5
# Returns list of points (effective) with their mass
function evaluate(cell, x, y, theta=.5)
    
    if cell.nbr_of_points == 0
        return []
    end
    
    center_of_mass_x = cell.sum_of_points_x / cell.nbr_of_points
    center_of_mass_y = cell.sum_of_points_y / cell.nbr_of_points

    distance = get_distance(x, y, center_of_mass_x, center_of_mass_y)
    angle = (cell.w + cell.h)/2/distance
    
    if angle < theta || cell.nbr_of_points == 1
        return [(center_of_mass_x, center_of_mass_y, cell.nbr_of_points), ]
    else
        return [p for quad in cell.childrens for p in evaluate(quad, x, y, theta)]
        end;
    
    end;



In [91]:
eff_points = evaluate(root, .3, .3);
println(length(eff_points), " / ",root.nbr_of_points)

95 / 604


In [70]:
for _ in 1:100
    x, y = rand(2)
    insert!(root, x, y)
    end;

In [65]:
root.nbr_of_points

304

In [92]:
@benchmark evaluate(root, .3, .3)

BenchmarkTools.Trial: 
  memory estimate:  129.53 KiB
  allocs estimate:  3523
  --------------
  minimum time:     357.226 μs (0.00% GC)
  median time:      369.987 μs (0.00% GC)
  mean time:        410.575 μs (8.03% GC)
  maximum time:     12.716 ms (95.28% GC)
  --------------
  samples:          10000
  evals/sample:     1